In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Loading the libraries

In [ ]:
from sklearn.svm import SVR
from catboost import CatBoostRegressor as cb,Pool
from sklearn.metrics import mean_absolute_error
from xgboost import XGBRegressor
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score


In [ ]:
#Loading the data and looking at the data
train= pd.read_csv('/kaggle/input/tabular-playground-series-jul-2021/train.csv')
train.head()

In [ ]:
#checking the null data
train.isnull().sum()

In [ ]:
# it is full functional data without any null variables.

In [ ]:
# lets look at the dtype and the data
train.info()

In [ ]:
# our target variable are target_benzene,target_carbon_monoxide and target_nitrogen_oxides.

In [ ]:
# now lets change the date_time from object to float
train['date_time'] = train['date_time'].astype('datetime64[ns]').astype(np.int64)/10**9

In [ ]:
# lets look at the date_time 
train['date_time']

In [ ]:
#so now date_time is  usable as a  feature 

# selecting the features

In [ ]:

features= ['relative_humidity','sensor_1','sensor_2','absolute_humidity','sensor_3','sensor_4','sensor_5','deg_C','date_time']
x= train[features]
x

In [ ]:
y= train['target_carbon_monoxide']

In [ ]:
from sklearn.feature_selection import mutual_info_regression

def make_mi_scores(x,y):
    mi_scores = mutual_info_regression(x,y)
    mi_scores = pd.Series(mi_scores, name="MI Scores", index=x.columns)
    mi_scores = mi_scores.sort_values(ascending=False)
    return mi_scores

mi_scores = make_mi_scores(x,y)
mi_scores

In [ ]:
#mi scores show the relation btw variables and target
# you can do it for all the target features which i didnt to save time because data is highly correlated with target features

In [ ]:

def plot_mi_scores(scores):
    scores = scores.sort_values(ascending=True)
    width = np.arange(len(scores))
    ticks = list(scores.index)
    plt.barh(width, scores)
    plt.yticks(width, ticks)
    plt.title("Mutual Information Scores")


plt.figure(dpi=100, figsize=(8, 5))
plot_mi_scores(mi_scores)

In [ ]:
import seaborn as sns
sns.relplot(x="sensor_2", y="target_carbon_monoxide", data=train);

In [ ]:
# it shows a strong relationshio btw sensor 2 Nd carbon_monoxide

In [ ]:
sns.lmplot(x="relative_humidity", y="target_carbon_monoxide", data=train);

In [ ]:
# even with the lowest feature it shows a strong relationship

# now we will select the target

In [ ]:
# lets put the other  two target in different variables
benzene= train['target_benzene']
nitrogen= train['target_nitrogen_oxides']

# splitting the data (train and test)

In [ ]:
from sklearn.model_selection import train_test_split
train_X, val_X, train_y,val_y = train_test_split(x,y, test_size=0.33, random_state=0)

In [ ]:
############IMPORTANT#########
# we will create 3 models for prediction and then add these models then divide by 3 to get the final prediction for 1 target variable
# this trick will give you a slightly better model than single model.

In [ ]:
rf= RandomForestRegressor()
rf.fit(train_X,train_y)
predictions= rf.predict(val_X)
print("mean ab error: "+ str(mean_absolute_error(val_y,predictions) ))

In [ ]:


xgb = XGBRegressor(n_estimators=600,n_jobs=8,learning_rate=0.1)
xgb.fit(train_X, train_y,early_stopping_rounds=5,eval_set=[(val_X,val_y)],verbose=False)
from sklearn.metrics import mean_absolute_error

predictions = xgb.predict(val_X)
print("Mean Absolute Error: " + str(mean_absolute_error(predictions, val_y)))

In [ ]:
train_dataset = Pool(train_X, train_y)



In [ ]:
cbr = cb(loss_function='RMSE')
 
grid = {'iterations': [100, 150, 200],
        'learning_rate': [0.03, 0.1],
        'depth': [2, 4, 6, 8],
        'l2_leaf_reg': [0.2, 0.5, 1, 3]}
cbr.grid_search(grid,train_dataset )

In [ ]:
cbr_pred = cbr.predict(val_X)
rmse = (np.sqrt(mean_squared_error(val_y, pred)))
r2 = r2_score(val_y, pred)
print('Testing performance')
print('RMSE: {:.2f}'.format(rmse))
print('R2: {:.2f}'.format(r2))

In [ ]:
print("Mean Absolute Error: " + str(mean_absolute_error(val_y,cbr_pred)))


In [ ]:
train_target_carbon_monoxide= (xgb.predict(x)+cbr.predict(x)+rf.predict(x))/3

In [ ]:
train_target_carbon_monoxide

In [ ]:
# it is doing good in the test so we will move on to the test predictions

In [ ]:
# we got the values of target_carbon_monoxide it is doing good in the training so we are going to do in the test
test = pd.read_csv('/kaggle/input/tabular-playground-series-jul-2021/test.csv')
test.head()

In [ ]:
# copying test-date_time for submission
test_date_time = test['date_time'].copy()

In [ ]:
# changing dates for making it usable
test['date_time'] = test['date_time'].astype('datetime64[ns]').astype(np.int64)/10**9


In [ ]:
X = test[features]
X

In [ ]:
# now predicting the values of test set of target carbon_mono_oxide
target_carbon_monoxide= (xgb.predict(X)+rf.predict(X)+cbr.predict(X))/3

In [ ]:
target_carbon_monoxide

In [ ]:
# we  got the first target values ,and the values are just fine
# lets move on to second and third target


In [ ]:
# new target feature benzene , 
y= benzene 

In [ ]:
# we have to do the same as above
# first splitting data into train and test

In [ ]:
train_X, val_X, train_y,val_y = train_test_split(x,y, test_size=0.3, random_state=0)

In [ ]:
# now creating and fitting the models`

#  Random forest model

In [ ]:
# hyper parameter tuning
from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
print(random_grid)

In [ ]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random.fit(train_X, train_y)

In [ ]:
rf_random.best_params_

In [ ]:
rf = RandomForestRegressor(n_estimators = 1000,min_samples_split= 2,min_samples_leaf= 1,max_features= 'sqrt',max_depth= 20,bootstrap= True)
rf.fit(train_X, train_y)

In [ ]:
predictions= rf.predict(val_X)
print("mean ab error: "+ str(mean_absolute_error(val_y,predictions) ))

## XGboost Regressor Model

In [ ]:
xgb = XGBRegressor(n_estimators=600,n_jobs=8,learning_rate=0.1)
xgb.fit(train_X, train_y,early_stopping_rounds=5,eval_set=[(val_X,val_y)],verbose=False)
from sklearn.metrics import mean_absolute_error

predictions = xgb.predict(val_X)
print("Mean Absolute Error: " + str(mean_absolute_error(predictions, val_y)))

#  catboost regressor model

In [ ]:
cbr = cb(iterations=1000,
                             learning_rate=0.02,
                             depth=12,
                             bagging_temperature = 0.2,
                            verbose=False,
                             od_wait=100)
cbr.fit(train_X,train_y)
predictions = cbr.predict(val_X)
print("Mean Absolute Error: " + str(mean_absolute_error(predictions, val_y)))

In [ ]:
# let's check ☑ training predictions now

In [ ]:
# we didn't choose random forest this time because model was not doing much well , so we opt for that only
train_benzene= (cbr.predict(x)+rf.predict(x))/2

In [ ]:
train_benzene

In [ ]:
# well train data does somewhat fine on predicting the values , so lets move on to target prediction

In [ ]:
# now we will do it in test data
# we are choosing only catboost and xgboost because random forest not doing much good prediction in this case
target_benzene = (rf.predict(X)+cbr.predict(X)+xgb.predict(X))/3

In [ ]:
target_benzene

In [ ]:
#we will move to our last target

In [ ]:
y=nitrogen

In [ ]:
train_X, val_X, train_y, val_y = train_test_split(x, y, random_state=1)

# RF MODEL

In [ ]:
# hyper parameter tuning
from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
print(random_grid)
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random.fit(train_X, train_y)

In [ ]:
rf_random.best_params_

In [ ]:
rf = RandomForestRegressor(n_estimators = 400,min_samples_split= 2,min_samples_leaf=1 ,max_features= 'sqrt',max_depth= None,bootstrap= False)
rf.fit(train_X, train_y)

In [ ]:
predictions= rf.predict(val_X)
print("mean ab error: "+ str(mean_absolute_error(val_y,predictions) ))

#  XG BOOST MODEL-

In [ ]:
from xgboost import XGBRegressor

xgb = XGBRegressor(n_estimators=600,n_jobs=8,learning_rate=0.1)
xgb.fit(train_X, train_y,early_stopping_rounds=100,eval_set=[(val_X,val_y)],verbose=False)
from sklearn.metrics import mean_absolute_error

predictions = xgb.predict(val_X)
print("Mean Absolute Error: " + str(mean_absolute_error(predictions, val_y)))

#  CAT MODEL

In [ ]:
train_dataset = Pool(train_X, train_y)



In [ ]:
cbr = cb(loss_function='RMSE')
 
grid = {'iterations': [100, 150, 200],
        'learning_rate': [0.03, 0.1],
        'depth': [2, 4, 6, 8],
        'l2_leaf_reg': [0.2, 0.5, 1, 3]}
cbr.grid_search(grid,train_dataset )

In [ ]:
cbr_pred = cbr.predict(val_X)
rmse = (np.sqrt(mean_squared_error(val_y, cbr_pred)))
r2 = r2_score(val_y, cbr_pred)
print('Testing performance')
print('RMSE: {:.2f}'.format(rmse))
print('R2: {:.2f}'.format(r2))

In [ ]:
print("Mean Absolute Error: " + str(mean_absolute_error(val_y,cbr_pred)))


In [ ]:
# to check how close this prediction is to the real data

train_nitrogen= (cbr.predict(x)+xgb.predict(x)+rf.predict(x))/3

In [ ]:
# to check how close this prediction is to the real data

In [ ]:
# model is doing good lets do it in target variable

In [ ]:
target_nitrogen_oxides= (cbr.predict(X)+xgb.predict(X)+rf.predict(X))/3

In [ ]:
target_nitrogen_oxides

# Now, submitting the file.

In [ ]:
submission= pd.DataFrame({"date_time": test_date_time,
                         "target_carbon_monoxide":target_carbon_monoxide,
                          "target_benzene":target_benzene,
                          "target_nitrogen_oxides":target_nitrogen_oxides})
submission.to_csv('submission.csv',index=False)